# api

This is the primary interface to running squ wrappers 

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from squ.core import *
from diskcache import memoize_stampede
import pandas

## List Workspaces

The `list_workspaces` function retreives a list of workspaces from blob storage and returns it in various formats

In [ ]:
#| exports
@memoize_stampede(cache, expire=60 * 60 * 3) # cache for 3 hours
def list_workspaces(fmt: str = "df", # df, csv, json, list
                    agency: str = "ALL"): # Agency alias or ALL
    path = datalake_path()
    df = pandas.read_csv((path / "notebooks/lists/SentinelWorkspaces.csv").open())
    df = df.join(pandas.read_csv((path / "notebooks/lists/SecOps Groups.csv").open()).set_index("Alias"), on="SecOps Group", rsuffix="_secops")
    df = df.rename(columns={"SecOps Group": "alias", "Domains and IPs": "domains"})
    df = df.dropna(subset=["customerId"]).sort_values(by="alias")
    if agency != "ALL":
        df = df[df["alias"] == agency]
    if fmt == "df":
        return df
    elif fmt == "csv":
        return df.to_csv()
    elif fmt == "json":
        return df.fillna("").to_dict("records")
    elif fmt == "list":
        return list(df["customerId"].unique())
    else:
        raise ValueError("Invalid format")

In [ ]:
list_workspaces()

,DIRECTORY,ITSA Email,Information Classification,JiraOrgId,LOCATION,M365 Security Portal,RESOURCE GROUP,alias,SecOps Status,Subscription Name,Title,customerId,subscription,AS Numbers,Agency Profile,Code Name,Current SLA,domains,Email Identities,Full Time Employee,ITSA Email_secops,JiraOrgId_secops,Primary Agency,Primary Agency Type,Risk Profile,SOC Intent,Status,Target SLA
39,Aqwest,it_alerts@aqwest.com.au,OFFICIAL,36.0,Australia Central,NaN,rg_aqwestcorp_azureau_central,Aqwest,Connected:T0; Risk:High - Critical Infrastructure,Aqwest Bunbury Water Corp Azure,Sentinel-Aqwest,a39833b7-4916-411f-a1fe-104f5f33e18e,d59fb7e1-3aed-490f-8ae4-9dba13c8d4dc,NaN,NaN,Drovers,T0,aqwest.com.au,NaN,42.0,it_alerts@aqwest.com.au,36.0,Aqwest,Schedule 1 Entity,High - Critical Infrastructure,Connected,Connected:T0; Risk:High - Critical Infrastructure,NaN
47,Construction Training Fund,emoore@bcitf.org,OFFICIAL,49.0,Australia East,NaN,ctf-prd-sentinel-rg,CTF,Connected:T0; Risk:Low,CTF - PROD,ctf-prd-sentinel-law,16ab370e-3337-44f8-9f95-78a36f1aa4de,3367a1cc-7809-493a-8acf-dbf0dd1db823,NaN,NaN,Great Sandy,T0,NaN,jbertram@ctf.wa.gov.au,30.0,emoore@bcitf.org,49.0,Construction Training Fund,Non-SES Entity,Low,Connected,Connected:T0; Risk:Low,SLA1
31,ChemCentre,itsa@chemcentre.wa.gov.au,OFFICIAL,32.0,Australia East,NaN,ccwa_au_ea_rg1,ChemCentre,Connected:T0; Risk:Medium,PAYG-PG_CC,CCWA-LogAnalytics-Sentinel,b989c5cc-7f77-4d47-945d-4861b4b3660f,747cd552-77da-43eb-8f85-9e251a0e106f,NaN,NaN,Coalseam,T0,ccwa.wa.gov.au\nchemcentre.wa.gov.au\nchemcent...,NaN,140.0,itsa@chemcentre.wa.gov.au,32.0,Chemistry Centre (WA),SES Organisation (Schedule 2),Medium,Connected,Connected:T0; Risk:Medium,NaN
0,"Department of Biodiversity, Conservation and A...",ITSA@dbca.wa.gov.au,OFFICIAL,3.0,Australia Southeast,https://security.microsoft.com/v2/advanced-hun...,oim-appservices,DBCA,Connected:T1; Risk:Medium,OIM Azure Subscription,azlogs01,e0cbfa81-08ce-4552-bbbb-76b385d040b3,bc1b9b1f-f768-42b8-b6b4-c7841269e454,NaN,NaN,Leeuwin,T1,aspectsofkingspark.com\naspectsofkingspark.com...,"chris.hocking@dbca.wa.gov.au, brendan.cale@dbc...",2046.0,ITSA@dbca.wa.gov.au,3.0,"Department of Biodiversity, Conservation and A...",Department (Section 35),Medium,Connected,Connected:T1; Risk:Medium,SLA1
8,Department of Fire and Emergency Services,ictsa@dfes.wa.gov.au,OFFICIAL,22.0,Australia East,NaN,azaue-sentinel-prod-01-rg,DFES,Connected:T0; Risk:High - Emergency Services,DFES-Production-EA,DFES-Sentinel-Log-Analytics,68e1aeec-3508-422a-a147-183a4e7e916d,3e8226f2-8828-4a79-935e-fc6fa71b37dd,NaN,NaN,One Tree Bridge,T0,dfes.wa.gov.au\ndfeswa.onmicrosoft.com\nfesa.w...,sami.anderson@dfes.wa.gov.au\nsimon.rice@dfes....,1673.0,ictsa@dfes.wa.gov.au,22.0,Department of Fire and Emergency Services,Department (Section 35),High - Emergency Services,Connected,Connected:T0; Risk:High - Emergency Services,NaN
13,"Department of Local Government, Sport and Cult...",ITSA@dlgsc.wa.gov.au,OFFICIAL,8.0,Australia Southeast,NaN,dlgsc-rg-prod-la-ause,DLGSCI,Connected:MOU (SLA0); Risk:Medium,Prod IaaS,dlgsc-laws-prod-001,567b2cd3-ae73-47f6-a114-03d28b782fdc,69b326d8-d5e8-464e-9397-f27b9a409518,NaN,NaN,Dampier,MOU (SLA0),artgallery.wa.gov.au\nartsculturetrust.wa.gov....,"majid.bassiri@dlgsc.wa.gov.au, vincent.law@dlg...",943.0,ITSA@dlgsc.wa.gov.au,8.0,"Department of Local Government, Sport and Cult...",Department (Section 35),Medium,Connected,Connected:MOU (SLA0); Risk:Medium,NaN
30,Department of Communities,itsa@communities.wa.gov.au,OFFICIAL,12.0,Australia East,NaN,doc-cor-arg-monitoring,DOC,Connected:SLA1; Risk:Medium,DoC-IT-Platform-LZ,doc-syd-cor-oms-002,42328089-3f38-492c-9cc1-fbb76847418f,cbe2a203-938a-41d2-a2be-e8806d402bd0,NaN,NaN,Kalbarri,SLA1,communities.wa.gov.au\ndhw.wa.gov.au\ndsc.wa.g...,"Glen.Bousfield@communities.wa.gov.au, Peter.To...",5113.0,itsa@communities.wa.gov.au,12.0,Department of Communities,Department (Section 35),Medium,Connected,Connected:SLA1; Risk:Medium,NaN
53,Department of Education,itsa

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()